- classifies the worldview imagery
- looks at the dataframe and creates columns for mpf and sic

In [3]:
import rasterio
from pylab import *
import numpy as np
import sys, os
import pandas as pd
import h5py
from PIL import Image
import shutil

In [4]:
def peakdet(v, delta, x = None):
    
    maxtab = []
    mintab = []
       
    if x is None:
        x = np.arange(len(v))
    
    v = np.asarray(v)
    
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    
    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    
    lookformax = True
   # mintab.append((0, 0.))
    
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True
    return array(maxtab), array(mintab)

In [3]:
# # move all WV images

# for a in [a for a in os.listdir('/Volumes/DataEllen/WorldView/2020_summer/') if a.endswith('.tif')]:
#     shutil.move('/Volumes/DataEllen/WorldView/2020_summer/'+a,'/Volumes/DataEllen/WorldView/2020_summer/'+'tif/'+a)

In [1]:
direc='/Users/buckley/Documents/DATA/Worldview/2020_summer/tif/multi/'

In [13]:
pwd

'/Users/buckley/Documents/ANALYSIS/Results/WV'

In [14]:
[a for a in os.listdir(direc) if a.endswith('.tif')]

['WV03_20190731005649_1040010050CC3B00_19JUL31005649-M1BS-503425895070_01_P001_u16ns3413.tif',
 'WV02_20190726005900_10300100961DFF00_19JUL26005900-M1BS-503429354020_01_P001_u16ns3413.tif',
 'WV02_20190707210531_10300100960A1800_19JUL07210531-M1BS-503347745020_01_P001_u16ns3413.tif',
 'WV02_20190724_clipped.tif',
 'WV02_20190726224200_1030010096A86400_19JUL26224200-M1BS-503429353100_01_P001_u16ns3413.tif',
 'WV02_20190804221208_1030010095038C00_19AUG04221208-M1BS-503581606060_01_P001_u16ns3413.tif',
 'WV02_20190724235615_1030010095212900_19JUL24235615-M1BS-503429305080_01_P001_u16ns3413.tif']

In [16]:
[a for a in os.listdir(direc) if '201908' in a]

['WV02_20190804221208_1030010095038C00_19AUG04221208-M1BS-503581606060_01_P001_u16ns3413.tif']

In [7]:
[a for a in os.listdir(direc) if a.startswith('WV02_20200903222726')]

['WV02_20200903222726_10300100AC613A00_20SEP03222726-M1BS-504713078040_01_P001_u16ns3413.tif']

In [19]:
#Oliwia you will have to change the directory to location of the files!
#direc='/Volumes/DataEllen/WorldView/2020_summer/tif/multi/'
#direc='/Volumes/DataEllen/WorldView/2019_summer/'
#direc='/Users/buckley/Documents/DATA/Worldview/2019_summer/multi/'
#direc='/Users/buckley/Documents/DATA/Worldview/2020_summer/tif/multi/'

if (os.path.exists('classification_hdf/')==False):
    os.mkdir('classification_hdf/')

f1=open('results.txt','a')
log=open('errorlog.txt','a')

for base in [a for a in os.listdir(direc) if a.startswith('WV02_20200903222726')]:#os.listdir(direc):#[a for a in os.listdir(direc) if '202008' in a]:#[a for a in os.listdir(direc) if a.endswith('.tif')]:#for base in ['WV03_20200727222946_104001005D827600_20JUL27222946-M1BS-504533127050_01_P001_u16ns3413.tif']:#'WV02_20190726224201_1030010096A86400_19JUL26224201-M1BS-503429353100_01_P002_u16ns3413.tif']:#['WV02_20200719235930_10300100A8CDBD00_20JUL19235930-M1BS-504533253030_01_P001_u16ns3413.tif']:#os.listdir('/Volumes/DataEllen/WorldView/2020_summer/tif/multi/'):

    
#     try:

        im = rasterio.open(direc+base)

        red=im.read(5)
        green=im.read(3)
        blue=im.read(2)
        nir=im.read(7)

        size_m=np.shape(red)

        br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))

        red_c=np.array(red,copy= True)
        blue_c=np.array(blue,copy= True)
        green_c=np.array(green,copy= True)
        nir_c= np.array(nir,copy= True)

        ## determine border pixels
        border=red_c<10


        #or not
        sect_xmin,sect_xmax=0,size_m[0]
        sect_ymin,sect_ymax=0,size_m[1]

        ##### separate water and not water

        bins=np.arange(-.5,.5,.02)
        Cbr1_n, bins= np.histogram(br1[sect_xmin:sect_xmax,sect_ymin:sect_ymax][~np.isnan(br1[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])], bins)

        dx= 0.0005*sum(Cbr1_n)  # dx is 0.05% difference
        maxtab,mintab = peakdet(Cbr1_n,dx,x=None)

        if np.where(Cbr1_n==max(Cbr1_n))[0][0]==maxtab[0][0]: # if there is only one max- do the fwhm to determine the cut
            if maxtab[0,0]==0:
                loc_cut=0
            else:        
                fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (Cbr1_n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
                loc_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

        else: #otherwise it is the minimum
            loc_cut=int(mintab[np.where(maxtab[:,0]==np.where(Cbr1_n==max(Cbr1_n))[0][0])[0][0]-1,0])

        if len(mintab)>0:
            br1_cut =bins[int(mintab[-1,0])]
        else:
            br1_cut=bins[loc_cut]
        water_mask=(br1>br1_cut)&(~border)


        #### classify ice
        binz=np.arange(0,1400,20)
        n,bins=np.histogram(red_c[sect_xmin:sect_xmax,sect_ymin:sect_ymax][(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~water_mask[sect_xmin:sect_xmax,sect_ymin:sect_ymax])].flatten(),bins=binz)
        dx= 0.0001*sum(n)  # dx is 0.01% difference
        maxtab,mintab = peakdet(n,dx,x=None)
        #print (maxtab, mintab)

        if np.where(n==max(n))[0][0]==maxtab[0][0]: # if there is only one max or first max is highest
            if maxtab[0,0]==0:
                bin_cut=0
            else:      
                fwhm= np.max(np.where((bins[:-1]<bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
                bin_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

        else:
            bin_cut=int(mintab[np.where(maxtab[:,0]==np.where(n==max(n))[0][0])[0][0]-1,0])

        ice_cut=bins[bin_cut]
        #print (ice_cut)

        ice_mask=(red_c>ice_cut)&(~border)&(~water_mask)
        other_mask=(red_c<=ice_cut)&(~border)&(~water_mask)


        ####### classify ow v melt pond

        binz=np.arange(0,1400,20)
        #using segment of image that is just sea ice so that it knows the true open water value
#         seg=np.zeros(size_m)
#         seg[3000:4000,5000:6000]=1
#         seg=seg.astype(bool)
        #n,bins,patches=plt.hist(blue_c[(~ice_mask)&(~border)&seg&(~other_mask)].flatten(),bins=binz)
        #n,bins,patches=plt.hist(blue_c[sect_xmin:sect_xmax,sect_ymin:sect_ymax][water_mask[sect_xmin:sect_xmax,sect_ymin:sect_ymax]].flatten(),bins=binz)
        # n,bins=np.histogram(blue_c[sect_xmin:sect_xmax,sect_ymin:sect_ymax][water_mask[sect_xmin:sect_xmax,sect_ymin:sect_ymax]].flatten(),bins=binz)
        # dx= 0.0001*sum(n)  # dx is 0.01% difference
        # maxtab,mintab = peakdet(n,dx,x=None)
        # #print (maxtab, mintab)
        # if (len(maxtab)>1):
        #     ow_cut=bins[int(mintab[-1,0])]
        #     #print (ow_cut)
        # elif bins[int(maxtab[0,0])]<200:
        #     n, bins= np.histogram(blue_c[water_mask], bins)
        #     fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.))))
        #     loc_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0]))
        #     ow_cut=bins[loc_cut]
        # else:
        #     ow_cut=100
        ow_cut=600
        ow_mask=(blue_c<ow_cut)&(water_mask)
        mp_mask=(blue_c>=ow_cut)&(water_mask)

        #stats!

        ow_pix=sum(ow_mask)
        mp_pix=sum(mp_mask)
        ice_pix=sum(ice_mask)
        border_pix=sum(border)
        other_pix=sum(other_mask)
        im_pix= (sect_xmax-sect_xmin)*(sect_ymax-sect_ymin)

        f1.write(base+'\t'+str(im_pix)+'\t'+str(border_pix)+'\t'+str(ice_pix)+'\t'+str(ow_pix)+'\t'+str(mp_pix)+'\t'+str(other_pix)+'\n')
        print((base+'\t'+str(im_pix)+'\t'+str(border_pix)+'\t'+str(ice_pix)+'\t'+str(ow_pix)+'\t'+str(mp_pix)+'\t'+str(other_pix)+'\n'))
        
        #check sum for pixels
        pixel_sum = (1 * border + 1 * ice_mask + 1 * ow_mask + 1 * mp_mask + 1 * other_mask)
        if (~np.all(pixel_sum == 1)):
            log.write("Sum pixels not 1 {0}: {1}\n".format(str(base), str(e)))
        
        # calculated parameters
        MPF=np.nan
        SIC=np.round((mp_pix+ice_pix)/float(ice_pix+mp_pix+ow_pix)*100,2)
        if SIC>15:
            MPF=np.round((mp_pix)/float(ice_pix+mp_pix)*100,2)

        #set up dataset for hdf5 files
        classification = np.zeros_like(border, dtype=np.int8)    # 8 bit int
        classification += (1 * ice_mask + 2 * ow_mask + 3 * mp_mask + 4 * other_mask)

        fout_name = 'classification_hdf/{}_classification.h5'.format(base)
        with h5py.File(fout_name, "w") as fout:
            fout.attrs["title"] = "Classification of Sentinel-2 Sea Ice Summer Melt Features"
            fout.attrs["creator_email"] = "buckley@umd.edu"
            
            # full reference when published
            fout.attrs["references"] = "..."
            fout.attrs["source_image"] = base
            fout.attrs["MPF (%)"] = MPF
            fout.attrs["SIC (%)"] = SIC


            dset = fout.create_dataset("classification", data=classification,
                    compression="gzip")
            dset.attrs["categories"] = "0 border, 1 ice, 2 open water, 3 melt pond, 4 other"
        
        
    # except Exception as e:
    #     log.write("Failed to process {0}: {1}\n".format(str(base), str(e)))
    # finally:
    #     pass
f1.close()
log.close()

/var/folders/y4/08xlrtkj0f11yqwtgkky7md00000gp/T/ipykernel_54012/4256329879.py:27: RuntimeWarning: invalid value encountered in true_divide
  br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))


WV02_20200903222726_10300100AC613A00_20SEP03222726-M1BS-504713078040_01_P001_u16ns3413.tif	101185518	25207789	62703957	9480335	145826	3647611



In [25]:
#Oliwia you will have to change the directory to location of the files!
#direc='/Volumes/DataEllen/WorldView/2020_summer/tif/multi/'
#direc='/Volumes/DataEllen/WorldView/2019_summer/'
#direc='/Users/buckley/Documents/DATA/Worldview/2019_summer/multi/'
#direc='/Users/buckley/Documents/DATA/Worldview/2020_summer/tif/multi/'

if (os.path.exists('classification_hdf/')==False):
    os.mkdir('classification_hdf/')

for base in [[a for a in os.listdir(direc) if a.endswith('.tif')][16]]:#[a for a in os.listdir(direc) if a.endswith('.tif')]:#for base in ['WV03_20200727222946_104001005D827600_20JUL27222946-M1BS-504533127050_01_P001_u16ns3413.tif']:#'WV02_20190726224201_1030010096A86400_19JUL26224201-M1BS-503429353100_01_P002_u16ns3413.tif']:#['WV02_20200719235930_10300100A8CDBD00_20JUL19235930-M1BS-504533253030_01_P001_u16ns3413.tif']:#os.listdir('/Volumes/DataEllen/WorldView/2020_summer/tif/multi/'):

    im = rasterio.open(direc+base)

    red=im.read(5)
    green=im.read(3)
    blue=im.read(2)
    nir=im.read(7)

    size_m=np.shape(red)

    br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))

    red_c=np.array(red,copy= True)
    blue_c=np.array(blue,copy= True)
    green_c=np.array(green,copy= True)
    nir_c= np.array(nir,copy= True)

    ## determine border pixels
    border=red_c<10


    #or not
    sect_xmin,sect_xmax=0,size_m[0]
    sect_ymin,sect_ymax=0,size_m[1]

    ##### separate water and not water

    bins=np.arange(-.5,.5,.02)
    Cbr1_n, bins= np.histogram(br1[~np.isnan(br1)&(~border)], bins)

    dx= 0.0005*sum(Cbr1_n)  # dx is 0.05% difference
    maxtab,mintab = peakdet(Cbr1_n,dx,x=None)

    if np.where(Cbr1_n==max(Cbr1_n))[0][0]==maxtab[0][0]: # if there is only one max- do the fwhm to determine the cut
        if maxtab[0,0]==0:
            loc_cut=0
        else:        
            fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (Cbr1_n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
            loc_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

    else: #otherwise it is the minimum
        loc_cut=int(mintab[np.where(maxtab[:,0]==np.where(Cbr1_n==max(Cbr1_n))[0][0])[0][0]-1,0])

    if len(mintab)>0:
        br1_cut =bins[int(mintab[-1,0])]
    else:
        br1_cut=bins[loc_cut]
    water_mask=(br1>br1_cut)&(~border)


    #### classify ice
    binz=np.arange(0,1400,20)
    n,bins=np.histogram(red_c[sect_xmin:sect_xmax,sect_ymin:sect_ymax][(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~water_mask[sect_xmin:sect_xmax,sect_ymin:sect_ymax])].flatten(),bins=binz)
    dx= 0.0001*sum(n)  # dx is 0.01% difference
    maxtab,mintab = peakdet(n,dx,x=None)
    #print (maxtab, mintab)

    if np.where(n==max(n))[0][0]==maxtab[0][0]: # if there is only one max or first max is highest
        if maxtab[0,0]==0:
            bin_cut=0
        else:      
            fwhm= np.max(np.where((bins[:-1]<bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
            bin_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

    else:
        bin_cut=int(mintab[np.where(maxtab[:,0]==np.where(n==max(n))[0][0])[0][0]-1,0])

    ice_cut=bins[bin_cut]
    #print (ice_cut)

    ice_mask=(red_c>ice_cut)&(~border)&(~water_mask)
    other_mask=(red_c<=ice_cut)&(~border)&(~water_mask)


    ####### classify ow v melt pond

    binz=np.arange(0,1400,20)

    n,bins=np.histogram(blue_c.flatten(),bins=binz)
    dx= 0.0001*sum(n)  # dx is 0.01% difference
    maxtab,mintab = peakdet(n,dx,x=None)
    #print (maxtab, mintab)
    if (len(maxtab)>1):
        ow_cut=bins[int(mintab[-1,0])]
        #print (ow_cut)
    elif bins[int(maxtab[0,0])]<2000:
        n, bins= np.histogram(blue_c[water_mask], bins)
        fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.))))
        loc_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0]))
        ow_cut=bins[loc_cut]
    else:
        ow_cut=100

    ow_mask=(blue_c<ow_cut)&(water_mask)
    mp_mask=(blue_c>=ow_cut)&(water_mask)

    #stats!

    ow_pix=sum(ow_mask)
    mp_pix=sum(mp_mask)
    ice_pix=sum(ice_mask)
    border_pix=sum(border)
    other_pix=sum(other_mask)
    im_pix= (sect_xmax-sect_xmin)*(sect_ymax-sect_ymin)

    # f1.write(base+'\t'+str(im_pix)+'\t'+str(border_pix)+'\t'+str(ice_pix)+'\t'+str(ow_pix)+'\t'+str(mp_pix)+'\t'+str(other_pix)+'\n')
    print((base+'\t'+str(im_pix)+'\t'+str(border_pix)+'\t'+str(ice_pix)+'\t'+str(ow_pix)+'\t'+str(mp_pix)+'\t'+str(other_pix)+'\n'))

    #check sum for pixels
    pixel_sum = (1 * border + 1 * ice_mask + 1 * ow_mask + 1 * mp_mask + 1 * other_mask)
    if (~np.all(pixel_sum == 1)):
        log.write("Sum pixels not 1 {0}: {1}\n".format(str(base), str(e)))

    # calculated parameters
    MPF=np.nan
    SIC=np.round((mp_pix+ice_pix)/float(ice_pix+mp_pix+ow_pix)*100,2)
    if SIC>15:
        MPF=np.round((mp_pix)/float(ice_pix+mp_pix)*100,2)

    #set up dataset for hdf5 files
    classification = np.zeros_like(border, dtype=np.int8)    # 8 bit int
    classification += (1 * ice_mask + 2 * ow_mask + 3 * mp_mask + 4 * other_mask)

    fout_name = '{}_classification.h5'.format(base)
    with h5py.File(fout_name, "w") as fout:
        fout.attrs["title"] = "Classification of Sentinel-2 Sea Ice Summer Melt Features"
        fout.attrs["creator_email"] = "buckley@umd.edu"

        # full reference when published
        fout.attrs["references"] = "..."
        fout.attrs["source_image"] = base
        fout.attrs["MPF (%)"] = MPF
        fout.attrs["SIC (%)"] = SIC


        dset = fout.create_dataset("classification", data=classification,
                compression="gzip")
        dset.attrs["categories"] = "0 border, 1 ice, 2 open water, 3 melt pond, 4 other"

        
f1.close()


/var/folders/y4/08xlrtkj0f11yqwtgkky7md00000gp/T/ipykernel_42063/1841464973.py:21: RuntimeWarning: invalid value encountered in true_divide
  br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))


WV02_20200826021615_10300100AD5C3100_20AUG26021615-M1BS-504712968030_01_P001_u16ns3413.tif	113360280	47522655	64763732	0	201699	872194



In [34]:
f = h5py.File(fout_name, 'r')

In [35]:
classif=f['classification'][:]

In [36]:
f

<HDF5 file "WV03_20190731005649_1040010050CC3B00_19JUL31005649-M1BS-503425895070_01_P001_u16ns3413.tif_classification.h5" (mode r)>

In [58]:
import tables
tables.file._open_files.close_all()

In [11]:
df=pd.read_csv('results.txt',delimiter='\t',names=['tile','im','border','ice','ow','mp','other'])
df['mpf']=df.mp/(df.mp+df.ice)
df['sic']=(df.ice+df.mp)/(df.ice+df.mp+df.ow)
df['other_p']=df.other/(df.im-df.border)
df['mp_p']=df.mp/(df.im-df.border)